In [1]:
import scanpy as sc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
#import pickle
from scipy.spatial import cKDTree
from SCTransform import SCTransform

import warnings
import time
warnings.filterwarnings('ignore')
#export PYTHONWARNINGS='ignore:Multiprocessing-backed parallel loops:UserWarning'
os.environ["PYTHONWARNINGS"] = "ignore::UserWarning"



In [2]:
def create_directory(outputFolder):
    answer=os.path.isdir(outputFolder)
    if answer==True:
        pass
    else:
        os.mkdir(outputFolder)


def find_index(sp_genename,sc_genename):
    index_sc=[]
    index_sp=[]
    d={}
    for j in range(len(sc_genename)):
        name=sc_genename[j]
        d[name]=j

    for i in range(len(sp_genename)):
        name=sp_genename[i]
        try:
            d[name]
            flag=1
        except KeyError:
            flag=0
        if flag==1:
            index_sc.append(d[name])
            index_sp.append(i)
    return index_sp,index_sc


In [4]:
datapath='./../alldata/'
ad_sc_ori=sc.read_h5ad(datapath+'sc_liver_data.h5ad')
#ad_sp_ori=sc.read_h5ad(datapath+'spatial_quadrant.h5ad')
ad_sp_ori=sc.read(datapath+'Blank_genes_removed.csv').transpose()
#ad_sp_ori.var_names_make_unique()
#ad_sc_ori.var_names_make_unique()

sc.pp.filter_cells(ad_sp_ori, min_counts=2)

In [5]:
print(ad_sc_ori)
print(ad_sp_ori)

AnnData object with n_obs × n_vars = 185894 × 31053
AnnData object with n_obs × n_vars = 393286 × 347
    obs: 'n_counts'


In [6]:
sp_genename=ad_sp_ori.var_names.to_numpy()
sc_genename=ad_sc_ori.var_names.to_numpy()
index_sp,index_sc=find_index(sp_genename,sc_genename)

In [7]:
ad_sc=ad_sc_ori[:,index_sc].copy()
ad_sp=ad_sp_ori[:,index_sp].copy()

In [8]:
print(ad_sc)
print(ad_sp)

AnnData object with n_obs × n_vars = 185894 × 346
AnnData object with n_obs × n_vars = 393286 × 346
    obs: 'n_counts'


In [9]:
sct_ad_sp = SCTransform(ad_sp,min_cells=5,gmean_eps=1,n_genes=500,n_cells=None, #use all cells
                        bin_size=500,bw_adjust=3,inplace=False)
sct_ad_sp.write_h5ad('scTransform_spatial.h5ad')

In [10]:
celname1=ad_sc.obs_names.to_numpy()
celname2=ad_sp.obs_names.to_numpy()
s1=np.unique(celname1)
s2=np.unique(celname2)
print(len(celname1),len(celname2))
print(len(s1),len(s2))

185894 393286
185894 393286


In [11]:
sct_ad_sc = SCTransform(ad_sc,min_cells=5,gmean_eps=1,n_genes=500,n_cells=None, #use all cells
		                bin_size=500,bw_adjust=3,inplace=False)
sct_ad_sc.write_h5ad('scTransform_singleCell.h5ad')

In [12]:
celname1=sct_ad_sc.obs_names.to_numpy()
celname2=sct_ad_sp.obs_names.to_numpy()
s1=np.unique(celname1)
s2=np.unique(celname2)
print(len(celname1),len(celname2))
print(len(s1),len(s2))

185894 393286
185894 393286
